# **Feature Extraction Methods: Imbalanced Data Without Annotations**

- *Key Features*: [MFCCs, Mel-Spectrograms, Chroma Frequencies, CQT]
- *Key Manipulations*: [Varying Window Sizes, Normalization, Average Pooling (Compression)]
- *Process Assistence*: [Converting them to numpy arrays now, easy label access across features]
- *Conversion*: [To numpy arrays and pkl files]


In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import os
import time

# Libraries for audio
from IPython.display import Audio
import librosa
import librosa.display

# Training and Testing Split
from sklearn.model_selection import train_test_split

# for normalization & avgpooling features
import tensorflow as tf
# for preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
import IPython.display as ipd
from functools import partial

In [2]:
# Variabels to be reused
path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/audio_files' 
npy_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/train_audio_npy/' 
train_csv = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/train-not-annotated.csv' 
annotated_train_csv = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/train-annotated.csv'
not_annotated_splt = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/trainval-split/trainval.csv'
sr = 22050

In [3]:
trainval_data = pd.read_csv(not_annotated_splt)
train_data = trainval_data[trainval_data['set'] == 'tr']
val_data = trainval_data[trainval_data['set'] == 'val']

In [4]:
train_data['species'].value_counts()

species
Himantopus himantopus        43
Alcedo atthis                37
Tachybaptus ruficollis       36
Botaurus stellaris           32
Gallinula chloropus          30
Charadrius alexandrinus      30
Porphyrio porphyrio          29
Fulica atra                  28
Acrocephalus melanopogon     28
Ardea purpurea               24
Motacilla flava              20
Acrocephalus scirpaceus      20
Circus aeruginosus           20
Acrocephalus arundinaceus    19
Ixobrychus minutus           19
Ciconia ciconia              19
Dendrocopos minor            19
Coracias garrulus            13
Anas platyrhynchos           12
Anas strepera                 4
Name: count, dtype: int64

# **Creating a class to do the extraction**

In [5]:
class Extraction:
  def __init__(self, train_df, val_df, window_size, overlap=0.5, npy_path=npy_path, sr=sr, n_mels=128, n_mfcc=20, n_chroma=12, n_cqt=84, hoplength=256, features=['mfcc'], normalize=True, avgpool=False, augment=True):
    """
    Instantiate the Extraction class to extract features.

    Parameters:
      sr (int): Sample rate of the audio files.
      n_mfccs (int): Number of MFCCs to extract.
      n_mels (int): Number of Mel bands to extract.
      n_chroma (int): Number of chroma bins to use.
      n_cqt (int): Number of CQT bins to use.
      features (list): List of features to extract.
        accepted features: 'mfcc', 'chroma', 'cqt', 'melspectrogram'.
      normalize (bool): Whether to normalize the features.
      avgpool (bool): Whether to avgpool the features.
    """
    
    self.train_df = train_df
    self.val_df = val_df
    self.npy_path = npy_path
    self.window_size = window_size
    self.overlap = overlap
    self.sr = sr
    self.n_mels = n_mels
    self.n_mfcc = n_mfcc
    self.n_chroma = n_chroma
    self.n_cqt = n_cqt
    self.hoplength = hoplength

     # confirm features have been specified
    assert len(features) != 0, "Must Specify At Least One Feature In The Form Of A List."
    self.features = features

    self.accepted_feature = ['mfcc', 'chroma', 'cqt', 'melspectrogram']
    for feature in self.features:
      assert feature in self.accepted_feature, f"{feature} is not an accepted feature, only 'mfcc', 'chroma', 'cqt', 'melspectrogram' are accepted features."

    self.normalize = normalize
    self.avgpool = avgpool
    self.augment = augment

    print(f"Train DataFrame shape: {train_df.shape}")
    print(f"Validation DataFrame shape: {val_df.shape}")

    # extract train and val labels and features
    self.train_y, self.train_features, self.train_ids = self.feature_extraction(self.train_df, window_size=self.window_size)
    self.val_y, self.val_features, self.val_ids = self.feature_extraction(self.val_df, window_size=self.window_size)

    # process the features by average pooling
    self.train_features_2D, self.val_features_2D, self.train_features_1D, self.val_features_1D = self.process_features(self.train_features, self.val_features)

  def random_augmentation(self, audio):
        """
        Apply random augmentation to the audio: pitch shift, time stretch, or add noise.
        """
        aug_type = np.random.choice(['pitch_up', 'pitch_down', 'noise', 'stretch', 'shrink'])

        if aug_type == 'pitch_up':
            # Random pitch up (between 1 and 4 semitones)
            pitch_shift = np.random.uniform(1, 4)
            audio = librosa.effects.pitch_shift(audio, sr=self.sr, n_steps=pitch_shift)
        elif aug_type == 'pitch_down':
            # Random pitch down (between -1 and -4 semitones)
            pitch_shift = np.random.uniform(-4, -1)
            audio = librosa.effects.pitch_shift(audio, sr=self.sr, n_steps=pitch_shift)
        elif aug_type == 'noise':
            # Add random noise at a fraction of the original audio volume
            noise = self.generate_pink_noise(len(audio))
            audio = audio + 0.01 * np.random.uniform(0.2, 0.5) * np.max(audio) * noise
        elif aug_type == 'stretch':
            # Randomly stretch audio (speed up)
            stretch_factor = np.random.uniform(1.1, 1.5)
            audio = librosa.effects.time_stretch(audio, rate=stretch_factor)
        elif aug_type == 'shrink':
            # Randomly shrink audio (slow down)
            stretch_factor = np.random.uniform(0.8, 0.9)
            audio = librosa.effects.time_stretch(audio, rate=stretch_factor)

  def normalize_audio(self, audio):
    return (audio - np.min(audio)) / (np.max(audio) - np.min(audio))
  
  def generate_pink_noise(self, num_samples):
    white_noise = np.random.randn(num_samples)
    
    # Apply a filter to convert white noise into pink noise (1/f noise)
    X = np.fft.rfft(white_noise)
    S = np.arange(1, len(X) + 1)  # Frequency scaling
    pink_noise = np.fft.irfft(X / S)

    if len(pink_noise) < num_samples:
        # Pad with zeros if the length is less than num_samples
        pink_noise = np.pad(pink_noise, (0, num_samples - len(pink_noise)), mode='constant')
    elif len(pink_noise) > num_samples:
        # Trim if necessary
        pink_noise = pink_noise[:num_samples]
    
    return self.normalize_audio(pink_noise)
  
  def pad_with_noise(self, audio_data, window_length, window_samples):
    current_length = librosa.get_duration(y=audio_data, sr=self.sr)

    if current_length > window_length:
        return audio_data
    
    target_length_samples = int(window_length * sr) 
    current_length_samples = window_samples
    padding_length_samples = target_length_samples - current_length_samples

    assert target_length_samples == (current_length_samples+padding_length_samples)
    
    # Generate pink noise to pad with
    pink_noise = self.generate_pink_noise(padding_length_samples)
    padded_audio = np.concatenate([audio_data, pink_noise])
    # if len(padded_audio) < target_length_samples:
    #     padded_audio = np.append(padded_audio, self.generate_pink_noise(1))

    assert target_length_samples == len(padded_audio)
    
    return padded_audio

#-------------------------Feature Extraction---------------------------------------
  def extract_mfcc(self, window):
    mfcc = librosa.feature.mfcc(y=window, sr=self.sr, n_mfcc=self.n_mfcc, hop_length=self.hoplength)
    if self.normalize:
      return librosa.util.normalize(mfcc)
    else:
      return mfcc


  def extract_chroma(self, window):
    chroma = librosa.feature.chroma_stft(y=window, sr=self.sr, n_chroma=self.n_chroma, hop_length=self.hoplength)
    if self.normalize:
      return librosa.util.normalize(chroma)
    else:
      return chroma
   

  def extract_cqt(self, window):
    cqt = librosa.cqt(y=window, sr=sr, hop_length=self.hoplength, n_bins=self.n_cqt)
    cqt_db = librosa.amplitude_to_db(np.abs(cqt), ref=np.max)
    return cqt_db

  def extract_melspectrogram(self, window):
    mel = librosa.feature.melspectrogram(y=window, sr=self.sr, n_mels=self.n_mels, hop_length=self.hoplength)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    if self.normalize:
      return librosa.util.normalize(mel_db)
    else:
      return mel_db
    
  def avgpooling(self, train_X, val_X, n_time, n_features):
    """
    Average pooling the train and val features.

    Parameters:
      train_X (npy): Training feature array of shape (batch_size, n_features, n_time)
      val_X (npy): Validation feature array of shape (batch_size, n_features, n_time)
      n_time (int): Time axis
      n_features (int): Feature axis

    Returns:
      train_X (npy): Avgpooled training feature array of shape (batch_size, n_features)
      val_X (npy): Avgpooled validation feature array of shape (batch_size, n_features)
    """
    # Clear the Keras session
    tf.keras.backend.clear_session()
    
    # Create the Keras input layer with shape (n_features, n_time)
    input_layer = tf.keras.layers.Input(shape=(n_features, n_time))
    
    # Apply average pooling over the time axis (axis=-1) to reduce n_time
    avg_pool = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1))(input_layer)
    
    # Build the model
    pooling_model = tf.keras.models.Model(inputs=input_layer, outputs=avg_pool)

    # Use the model to apply average pooling on the training and validation features
    train_X = pooling_model.predict(train_X)
    val_X = pooling_model.predict(val_X)

    return train_X, val_X

    
  def process_features(self, train_features_dict, val_features_dict):
    train_copy = train_features_dict.copy()
    val_copy = val_features_dict.copy()
    for each in train_copy.keys():
      
      if each == 'mfcc':
        n_features=self.n_mfcc
      elif each == 'chroma':
        n_features=self.n_chroma
      elif each == 'cqt':
        n_features=self.n_cqt
      elif each == 'melspectrogram':
        n_features=self.n_mels
      
      train_feature = train_copy[each]
      val_feature = val_copy[each]

      if self.avgpool:
        train_copy[each], val_copy[each] = self.avgpooling(train_feature, val_feature, n_time=train_feature.shape[2], n_features=n_features)
      else:
        train_copy[each], val_copy[each] = train_copy[each], val_copy[each]
    
    return train_features_dict, val_features_dict, train_copy, val_copy
      

  def feature_extraction(self, dataframe, window_size):
    y = [] # To hold the labels
    ids = [] # To hold the audio name (going to use it to aggregate the results for one audio file)
    features_dict = {item: [] for item in self.features} # Create a key for each feature listed
    print(f"Number of rows in dataframe: {len(dataframe)}")
    for _, row in tqdm(dataframe.iterrows(), desc="Processing data", total=len(dataframe)):
          label = row['species']
          file_path = os.path.join(self.npy_path, row['filename_npy'])
          id = row['audio_name']

          try:
              audio = np.load(file_path)
          except FileNotFoundError:
              print(f"File not found: {file_path}")
              continue


          audio = self.normalize_audio(audio)
          if self.augment:
                audio = self.random_augmentation(audio)
          audio = self.pad_with_noise(audio, window_length=self.window_size, window_samples=len(audio))
          # print(len(sample))

          window_samples = int(window_size * self.sr)
          hop_samples = int(window_samples * (1 - self.overlap))  # For overlapping

          # Break the audio into windows with the specified overlap
          audio_windows = librosa.util.frame(audio, frame_length=window_samples, hop_length=hop_samples).T
          
          
          # display(label)
          
          for _, window in enumerate(audio_windows):
              
              y.append(label)
              ids.append(id)

              if len(window) < window_samples:
                  if len(window) < 512*2:
                     continue
                  else:
                      window = self.pad_with_noise(window, window_length=window_size)
              
              # Feature Extraction FR --------------------------------------------------------------------
              # dynatically call the extract_x function to extract the listed features
              for feature in self.features:
                extract = f"extract_{feature}"
                if hasattr(self, extract) and callable(func := getattr(self, extract)):
                  features_dict[feature].append(func(window))

          # cast lists to np arrays
    for each in features_dict.keys():
              features_dict[each] = np.array(features_dict[each])

    y = np.array(y)
    ids = np.array(ids)

          # If not using average pooling, return resized features
    return y, features_dict, ids

# **Trying to balance the training set**

In [10]:
# Step 1: Identify the majority class count
species_counts = train_data['species'].value_counts()
majority_class_count = species_counts.max()

# Step 2: Define the target for oversampling (0.7 times the majority class count)
target_count = int(majority_class_count * 0.7)

# Step 3: Separate majority and minority classes
df_majority = train_data[train_data['species'].map(species_counts) == majority_class_count]

# Step 4: Oversample the minority classes to the target count
oversampled_data = pd.DataFrame()

for species, count in species_counts.items():
    species_df = train_data[train_data['species'] == species]
    if count < target_count:
        species_upsampled = resample(species_df, 
                                     replace=True,     # Allow oversampling
                                     n_samples=target_count,  # Target count for each class
                                     random_state=42)  # For reproducibility
        oversampled_data = pd.concat([oversampled_data, species_upsampled])
    else:
        oversampled_data = pd.concat([oversampled_data, species_df])

# Combine with majority class data
train_data_balanced = pd.concat([oversampled_data])

# Check the new class distribution
print(train_data_balanced['species'].value_counts())

species
Himantopus himantopus        43
Alcedo atthis                37
Tachybaptus ruficollis       36
Botaurus stellaris           32
Circus aeruginosus           30
Anas platyrhynchos           30
Coracias garrulus            30
Dendrocopos minor            30
Ciconia ciconia              30
Ixobrychus minutus           30
Acrocephalus arundinaceus    30
Motacilla flava              30
Acrocephalus scirpaceus      30
Ardea purpurea               30
Acrocephalus melanopogon     30
Fulica atra                  30
Porphyrio porphyrio          30
Charadrius alexandrinus      30
Gallinula chloropus          30
Anas strepera                30
Name: count, dtype: int64


# **Average Pooling**

## **Window Size = 6s**

### **['melspectrogram', 'mfcc', 'chroma', 'cqt']**

In [16]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'cqt']

In [17]:
features = Extraction(train_data,
                      val_data,
                      window_size=6,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (482, 6)
Validation DataFrame shape: (133, 6)
Number of rows in dataframe: 482


Processing data: 100%|██████████| 482/482 [17:35<00:00,  2.19s/it]


Number of rows in dataframe: 133


Processing data: 100%|██████████| 133/133 [02:57<00:00,  1.33s/it]


205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 673us/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 634us/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 748us/step
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [18]:
train_y = features.train_y
val_y = features.val_y

display(train_y.shape)
display(val_y.shape)

(6543,)

(2028,)

In [19]:
# Avgpooled Features
train_features_1D = features.train_features_1D
for key in train_features_1D.keys():
  display(key)
  display(train_features_1D[key].shape)

'melspectrogram'

(6543, 128)

'mfcc'

(6543, 20)

'chroma'

(6543, 12)

'cqt'

(6543, 84)

In [20]:
# Avgpooled Features
val_features_1D = features.val_features_1D
for key in val_features_1D.keys():
  display(key)
  display(val_features_1D[key].shape)

'melspectrogram'

(2028, 128)

'mfcc'

(2028, 20)

'chroma'

(2028, 12)

'cqt'

(2028, 84)

In [21]:
# Not avgpooled Features
train_features_2D = features.train_features_2D
for key in train_features_2D.keys():
  display(key)
  display(train_features_2D[key].shape)

'melspectrogram'

(6543, 128, 517)

'mfcc'

(6543, 20, 517)

'chroma'

(6543, 12, 517)

'cqt'

(6543, 84, 517)

In [22]:
# Not avgpooled Features
val_features_2D = features.val_features_2D
for key in val_features_2D.keys():
  display(key)
  display(val_features_2D[key].shape)

'melspectrogram'

(2028, 128, 517)

'mfcc'

(2028, 20, 517)

'chroma'

(2028, 12, 517)

'cqt'

(2028, 84, 517)

In [23]:
train_ids = features.train_ids
val_ids = features.val_ids

### Encode Classes

In [24]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [15 15 15 ... 12 12 12]
Encoded validation labels: [15 15 15 ... 12 12 12]


In [25]:
# Avg Pooled

train_features_1D['label'] = train_y_encoded
val_features_1D['label'] = val_y_encoded

train_features_1D['id'] = train_ids
val_features_1D['id'] = val_ids

# Not Avg Pooled

train_features_2D['label'] = train_y_encoded
val_features_2D['label'] = val_y_encoded

train_features_2D['id'] = train_ids
val_features_2D['id'] = val_ids

In [26]:
merged_dict_1D = {'train': train_features_1D, 'val': val_features_1D}
merged_dict_1D

{'train': {'melspectrogram': array([[-0.01776731, -0.989827  , -0.99071366, ..., -0.8216415 ,
          -0.85376334, -0.9824129 ],
         [-0.01776753, -0.9898328 , -0.99071616, ..., -0.8170307 ,
          -0.84770477, -0.9777302 ],
         [-0.01776784, -0.989846  , -0.99073434, ..., -0.82816565,
          -0.8608326 , -0.98463976],
         ...,
         [-0.01781281, -0.5766479 , -0.54141074, ..., -0.87102264,
          -0.89623916, -0.99998677],
         [-0.01774438, -0.57157916, -0.5401701 , ..., -0.8684321 ,
          -0.89438707, -0.99997807],
         [-0.01779194, -0.5788874 , -0.54096264, ..., -0.8674551 ,
          -0.89504766, -0.999996  ]], dtype=float32),
  'mfcc': array([[-1.        , -0.10011531, -0.14837739, ...,  0.02309681,
           0.0204311 ,  0.04844239],
         [-1.        , -0.11325053, -0.18573974, ...,  0.02437621,
           0.02035097,  0.05108224],
         [-1.        , -0.10374827, -0.18394189, ...,  0.02396246,
           0.01970905,  0.04918307]

In [27]:
merged_dict_2D = {'train': train_features_2D, 'val': val_features_2D}
merged_dict_2D

{'train': {'melspectrogram': array([[[-1.73638668e-02,  0.00000000e+00, -8.11987277e-03, ...,
           -1.67526044e-02, -5.46729891e-03, -8.64551112e-04],
          [-1.93848714e-01, -2.12280467e-01, -2.69487441e-01, ...,
           -3.62844795e-01, -2.54802048e-01, -2.05738589e-01],
          [-2.68348396e-01, -2.86629409e-01, -3.42423439e-01, ...,
           -4.13956523e-01, -3.27738762e-01, -2.80380309e-01],
          ...,
          [-9.98975337e-01, -9.98841584e-01, -9.96024787e-01, ...,
           -9.24670994e-01, -9.80563939e-01, -9.91996884e-01],
          [-9.99578595e-01, -9.99459684e-01, -9.97734010e-01, ...,
           -9.48356807e-01, -9.88852501e-01, -9.95917022e-01],
          [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
           -1.00000000e+00, -1.00000000e+00, -1.00000000e+00]],
  
         [[-1.72525998e-02,  0.00000000e+00, -8.14473629e-03, ...,
           -1.66835915e-02, -5.44226915e-03, -8.53342586e-04],
          [-1.92598045e-01, -2.11414754e-01,

### Save the merged dictionary to a pkl

In [28]:
# Avrg Pooled
with open('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/NotAnnotated/Augmented/AveragePooled/split_features_6s_all_1D.pkl', 'wb') as file:
  pickle.dump(merged_dict_1D, file)
del file

In [29]:
# Not Avg Pooled
with open('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/NotAnnotated/Augmented/NotAveragePooled/split_features_6s_all_2D.pkl', 'wb') as file:
  pickle.dump(merged_dict_2D, file)
del file

## **Window Size = 3s**

### **['melspectrogram', 'mfcc', 'chroma', 'cqt']**

In [30]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'cqt']

In [31]:
features = Extraction(train_data,
                      val_data,
                      window_size=3,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (482, 6)
Validation DataFrame shape: (133, 6)
Number of rows in dataframe: 482


Processing data:   0%|          | 0/482 [00:00<?, ?it/s]c:\Users\thato\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
Processing data: 100%|██████████| 482/482 [25:58<00:00,  3.23s/it]  


Number of rows in dataframe: 133


Processing data: 100%|██████████| 133/133 [03:47<00:00,  1.71s/it]


429/429 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
429/429 ━━━━━━━━━━━━━━━━━━━━ 0s 517us/step
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 475us/step
429/429 ━━━━━━━━━━━━━━━━━━━━ 0s 458us/step
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step
429/429 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [32]:
train_y = features.train_y
val_y = features.val_y

display(train_y.shape)
display(val_y.shape)

(13725,)

(4231,)

In [33]:
# Avgpooled Features
train_features_1D = features.train_features_1D
for key in train_features_1D.keys():
  display(key)
  display(train_features_1D[key].shape)

'melspectrogram'

(13725, 128)

'mfcc'

(13725, 20)

'chroma'

(13725, 12)

'cqt'

(13725, 84)

In [34]:
# Avgpooled Features
val_features_1D = features.val_features_1D
for key in val_features_1D.keys():
  display(key)
  display(val_features_1D[key].shape)

'melspectrogram'

(4231, 128)

'mfcc'

(4231, 20)

'chroma'

(4231, 12)

'cqt'

(4231, 84)

In [35]:
# Not avgpooled Features
train_features_2D = features.train_features_2D
for key in train_features_2D.keys():
  display(key)
  display(train_features_2D[key].shape)

'melspectrogram'

(13725, 128, 259)

'mfcc'

(13725, 20, 259)

'chroma'

(13725, 12, 259)

'cqt'

(13725, 84, 259)

In [36]:
# Not avgpooled Features
val_features_2D = features.val_features_2D
for key in val_features_2D.keys():
  display(key)
  display(val_features_2D[key].shape)

'melspectrogram'

(4231, 128, 259)

'mfcc'

(4231, 20, 259)

'chroma'

(4231, 12, 259)

'cqt'

(4231, 84, 259)

In [37]:
train_ids = features.train_ids
val_ids = features.val_ids

### Encode Classes

In [38]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [15 15 15 ... 12 12 12]
Encoded validation labels: [15 15 15 ... 12 12 12]


In [39]:
# Avg Pooled

train_features_1D['label'] = train_y_encoded
val_features_1D['label'] = val_y_encoded

train_features_1D['id'] = train_ids
val_features_1D['id'] = val_ids

# Not Avg Pooled

train_features_2D['label'] = train_y_encoded
val_features_2D['label'] = val_y_encoded

train_features_2D['id'] = train_ids
val_features_2D['id'] = val_ids

In [40]:
merged_dict_1D = {'train': train_features_1D, 'val': val_features_1D}
merged_dict_1D

{'train': {'melspectrogram': array([[-0.01764554, -0.97810674, -0.97999614, ..., -0.82831436,
          -0.86151993, -0.9852102 ],
         [-0.0176447 , -0.97808164, -0.9799673 , ..., -0.82414514,
          -0.85394555, -0.982918  ],
         [-0.01764692, -0.97814673, -0.9800371 , ..., -0.81990373,
          -0.8500417 , -0.98001206],
         ...,
         [-0.01758811, -0.57715386, -0.5305197 , ..., -0.86590964,
          -0.8951364 , -0.9996821 ],
         [-0.01772401, -0.58169186, -0.5389059 , ..., -0.8691575 ,
          -0.89791137, -0.99996406],
         [-0.01760773, -0.57361674, -0.5377899 , ..., -0.8711317 ,
          -0.9001475 , -0.99997455]], dtype=float32),
  'mfcc': array([[-1.        , -0.07280675, -0.12869048, ...,  0.02178697,
           0.02212632,  0.04228346],
         [-1.        , -0.08856594, -0.15130392, ...,  0.02659525,
           0.01738   ,  0.04704501],
         [-1.        , -0.10337539, -0.15928885, ...,  0.02455455,
           0.01868342,  0.05395849]

In [41]:
merged_dict_2D = {'train': train_features_2D, 'val': val_features_2D}
merged_dict_2D

{'train': {'melspectrogram': array([[[-1.73638668e-02,  0.00000000e+00, -8.11987277e-03, ...,
           -1.30902734e-02, -1.34414493e-03, -6.30739005e-03],
          [-1.93848714e-01, -2.12280467e-01, -2.69487441e-01, ...,
           -3.06648046e-01, -2.30226904e-01, -1.96743473e-01],
          [-2.68348396e-01, -2.86629409e-01, -3.42423439e-01, ...,
           -3.76332015e-01, -3.03701609e-01, -2.71256447e-01],
          ...,
          [-9.98975337e-01, -9.98841584e-01, -9.96024787e-01, ...,
           -9.55386639e-01, -9.84975278e-01, -9.91272390e-01],
          [-9.99578595e-01, -9.99459684e-01, -9.97734010e-01, ...,
           -9.75634158e-01, -9.92564321e-01, -9.96132433e-01],
          [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
           -1.00000000e+00, -1.00000000e+00, -1.00000000e+00]],
  
         [[-1.73103306e-02,  0.00000000e+00, -8.10205285e-03, ...,
           -1.30698718e-02, -1.34197809e-03, -6.29818905e-03],
          [-1.93254337e-01, -2.11670950e-01,

### Save the merged dictionary to a pkl

In [42]:
# Avrg Pooled
with open('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/NotAnnotated/Augmented/AveragePooled/split_features_3s_all_1D.pkl', 'wb') as file:
  pickle.dump(merged_dict_1D, file)
del file

In [43]:
# Avrg Pooled
with open('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/NotAnnotated/Augmented/NotAveragePooled/split_features_3s_all_2D.pkl', 'wb') as file:
  pickle.dump(merged_dict_2D, file)
del file

## **Window Size = 1s**

### **['melspectrogram', 'mfcc', 'chroma', 'cqt']**

In [44]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'cqt']

In [45]:
features = Extraction(train_data,
                      val_data,
                      window_size=1,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (482, 6)
Validation DataFrame shape: (133, 6)
Number of rows in dataframe: 482


Processing data:   0%|          | 0/482 [00:00<?, ?it/s]c:\Users\thato\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
Processing data: 100%|██████████| 482/482 [33:42<00:00,  4.20s/it]  


Number of rows in dataframe: 133


Processing data: 100%|██████████| 133/133 [08:24<00:00,  3.79s/it]


1331/1331 ━━━━━━━━━━━━━━━━━━━━ 1s 619us/step
409/409 ━━━━━━━━━━━━━━━━━━━━ 0s 684us/step
1331/1331 ━━━━━━━━━━━━━━━━━━━━ 1s 474us/step
409/409 ━━━━━━━━━━━━━━━━━━━━ 0s 463us/step
1331/1331 ━━━━━━━━━━━━━━━━━━━━ 1s 458us/step
409/409 ━━━━━━━━━━━━━━━━━━━━ 0s 428us/step
1331/1331 ━━━━━━━━━━━━━━━━━━━━ 1s 566us/step
409/409 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step


In [46]:
train_y = features.train_y
val_y = features.val_y

display(train_y.shape)
display(val_y.shape)

(42578,)

(13083,)

In [47]:
# Avgpooled Features
train_features_1D = features.train_features_1D
for key in train_features_1D.keys():
  display(key)
  display(train_features_1D[key].shape)

'melspectrogram'

(42578, 128)

'mfcc'

(42578, 20)

'chroma'

(42578, 12)

'cqt'

(42578, 84)

In [48]:
# Avgpooled Features
val_features_1D = features.val_features_1D
for key in val_features_1D.keys():
  display(key)
  display(val_features_1D[key].shape)

'melspectrogram'

(13083, 128)

'mfcc'

(13083, 20)

'chroma'

(13083, 12)

'cqt'

(13083, 84)

In [49]:
# Not avgpooled Features
train_features_2D = features.train_features_2D
for key in train_features_2D.keys():
  display(key)
  display(train_features_2D[key].shape)

'melspectrogram'

(42578, 128, 87)

'mfcc'

(42578, 20, 87)

'chroma'

(42578, 12, 87)

'cqt'

(42578, 84, 87)

In [50]:
# Not avgpooled Features
val_features_2D = features.val_features_2D
for key in val_features_2D.keys():
  display(key)
  display(val_features_2D[key].shape)

'melspectrogram'

(13083, 128, 87)

'mfcc'

(13083, 20, 87)

'chroma'

(13083, 12, 87)

'cqt'

(13083, 84, 87)

In [51]:
train_ids = features.train_ids
val_ids = features.val_ids

### Encode Classes

In [52]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [15 15 15 ... 12 12 12]
Encoded validation labels: [15 15 15 ... 12 12 12]


In [53]:
# Avg Pooled

train_features_1D['label'] = train_y_encoded
val_features_1D['label'] = val_y_encoded

train_features_1D['id'] = train_ids
val_features_1D['id'] = val_ids

# Not Avg Pooled

train_features_2D['label'] = train_y_encoded
val_features_2D['label'] = val_y_encoded

train_features_2D['id'] = train_ids
val_features_2D['id'] = val_ids

In [54]:
merged_dict_1D = {'train': train_features_1D, 'val': val_features_1D}
merged_dict_1D

{'train': {'melspectrogram': array([[-0.01720456, -0.9333485 , -0.9391832 , ..., -0.82813597,
          -0.8620985 , -0.9794059 ],
         [-0.01720306, -0.9333132 , -0.93914014, ..., -0.8327555 ,
          -0.86637986, -0.97935015],
         [-0.01720168, -0.9332716 , -0.93908674, ..., -0.847576  ,
          -0.8784226 , -0.9896085 ],
         ...,
         [-0.01720948, -0.55143994, -0.52205884, ..., -0.87508094,
          -0.90587527, -0.9997576 ],
         [-0.0171332 , -0.55125844, -0.51631904, ..., -0.8850505 ,
          -0.9144034 , -0.99985427],
         [-0.01714398, -0.5486961 , -0.5132266 , ..., -0.88104194,
          -0.911118  , -1.        ]], dtype=float32),
  'mfcc': array([[-1.        , -0.02898505, -0.10768069, ...,  0.01827751,
           0.02722204,  0.04308926],
         [-1.        , -0.02902972, -0.1065027 , ...,  0.01701666,
           0.02487057,  0.04410085],
         [-1.        , -0.0125336 , -0.09841443, ...,  0.02036034,
           0.02327715,  0.04241924]

In [55]:
merged_dict_2D = {'train': train_features_2D, 'val': val_features_2D}
merged_dict_2D

{'train': {'melspectrogram': array([[[-1.7363867e-02,  0.0000000e+00, -8.1198728e-03, ...,
           -9.7965123e-03, -4.7687645e-05, -1.3045984e-02],
          [-1.9384871e-01, -2.1228047e-01, -2.6948744e-01, ...,
           -2.7984515e-01, -2.1688208e-01, -1.9360192e-01],
          [-2.6834840e-01, -2.8662941e-01, -3.4242344e-01, ...,
           -3.5257360e-01, -2.9082543e-01, -2.6788574e-01],
          ...,
          [-9.9897534e-01, -9.9884158e-01, -9.9602479e-01, ...,
           -9.7042859e-01, -9.8348904e-01, -9.8610806e-01],
          [-9.9957860e-01, -9.9945968e-01, -9.9773401e-01, ...,
           -9.7953582e-01, -9.8669928e-01, -9.8847580e-01],
          [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, ...,
           -1.0000000e+00, -1.0000000e+00, -1.0000000e+00]],
  
         [[-1.7287681e-02,  0.0000000e+00, -8.1161708e-03, ...,
           -9.7880140e-03, -4.7853238e-05, -1.3035558e-02],
          [-1.9298783e-01, -2.1179421e-01, -2.6936716e-01, ...,
           -2.7959031e

### Save the merged dictionary to a pkl

In [56]:
# Avrg Pooled
with open('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/NotAnnotated/Augmented/AveragePooled/split_features_1s_all_1D.pkl', 'wb') as file:
  pickle.dump(merged_dict_1D, file)
del file

In [57]:
# Avrg Pooled
with open('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/NotAnnotated/Augmented/NotAveragePooled/split_features_1s_all_2D.pkl', 'wb') as file:
  pickle.dump(merged_dict_2D, file)
del file